In [32]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler, scale
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split

In [33]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

# 데이터 불러오기 / 데이터 정리

In [34]:
Medicine_nonfinance = pd.read_excel('./data/dongboo/비재무/제약_비재무_TOTAL_최종.xlsx')
Medicine_val_nonfinance = pd.read_excel('./data/dongboo/비재무validation/validation_비재무_제약_total_최종.xlsx')

In [35]:
Medicine_stock = pd.read_excel('./data/dongboo/제약_주가_total_krx.xlsx')
Medicine_val_stock = pd.read_excel('./data/dongboo/제약_validation_주가.xls')

In [36]:
Medicine = pd.merge(Medicine_nonfinance,Medicine_stock,how = 'left', on = ['회사명','거래소코드','회계년도'])
Medicine_val = pd.merge(Medicine_val_nonfinance,Medicine_val_stock,how = 'left', on = ['회사명','거래소코드','회계년도'])

# 회사별로 분리

In [37]:
greencross = Medicine[Medicine['회사명'] == '(주)녹십자']
daewoong = Medicine[Medicine['회사명'] == '(주)대웅제약']
yoohan = Medicine[Medicine['회사명'] == '(주)유한양행']
hanmi = Medicine[Medicine['회사명'] == '한미약품(주)']

#---------- validation ---------------------

donga = Medicine_val[Medicine_val['회사명'] == '동아에스티(주)']
boogwang = Medicine_val[Medicine_val['회사명'] == '부광약품(주)']
ilyang = Medicine_val[Medicine_val['회사명'] == '일양약품(주)']
jw = Medicine_val[Medicine_val['회사명'] == '제이더블유중외제약(주)']

In [38]:
def classify(stock):
    
    # 회계년도 순으로 정렬 및 인덱스 초기화
    stock.sort_values('회계년도', ascending= True, inplace = True)
    
    stock.reset_index(drop = True, inplace = True)
    
    
    # 새로운 변수 추가
    
    stock['3개월후종가'] = stock['종가'][1:].reset_index(drop=True)
    
    stock['분기수익률'] = (stock['3개월후종가'] - stock['종가']) / stock['종가']
    
    stock['수익률상승하락'] = stock['분기수익률']
    
    for i in range(len(stock)):
        if stock['분기수익률'][i] < 0.005:
            stock['수익률상승하락'][i] = 0
        else:
            stock['수익률상승하락'][i] = 1
        
       
    return stock

In [39]:
classify(greencross)
classify(daewoong)
classify(yoohan)
classify(hanmi)
#---------- validation ---------------------
classify(donga)
classify(boogwang)
classify(ilyang)
classify(jw)

,회사명,거래소코드,회계년도,주식의 종류,발행할 주식의 총수(주),현재까지 발행한 주식의 총수(주),현재까지 감소한 주식의 총수(주),유통주식수 (발행주식의 총수 - 자기주식수)(주),이익소각(주),발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주),자기주식수(주),정규+무기계약_기말인원(남)(명),정규+무기계약_기말인원(여)(명),정규+무기계약_기말인원(명),기간제_기말인원(남)(명),기간제_기말인원(여)(명),기간제_기말인원(명),합계_기말인원_남(명),합계_기말인원_여(명),합계_기말인원(명),합계_년간평균급여(천원),합계_1인평균년간급여(천원),합계_평균근속연수,최고가,최저가,종가,거래량,거래대금,3개월후종가,분기수익률,수익률상승하락
0,제이더블유중외제약(주),1060,2010/06,10.0,24000000,14614391,5002876,9608445,0,9611515,3070,0,0,0,0,0,0,936,234,1170,24174000,21000,6.93,17700,15800,16750.0,1206805,21259162500,16425.0,-0.019403,0.0
1,제이더블유중외제약(주),1060,2010/09,10.0,24000000,14614391,5002876,9608445,0,9611515,3070,0,0,0,0,0,0,916,226,1142,37721000,32000,6.92,16650,16200,16425.0,546347,8951326750,15875.0,-0.033486,0.0
2,제이더블유중외제약(주),1060,2010/12,10.0,24000000,14614391,5002876,9608445,0,9611515,3070,908,228,1136,9,0,9,917,228,1145,51687000,45000,6.75,16450,15300,15875.0,558784,8898589050,13775.0,-0.132283,0.0
3,제이더블유중외제약(주),1060,2011/03,10.0,24000000,14902649,5002876,9893871,0,9899773,5902,891,230,1121,6,0,6,897,230,1127,12469000,11064,6.50,14550,13000,13775.0,519523,7138146800,13275.0,-0.036298,0.0
4,제이더블유중외제약(주),1060,2011/06,10.0,24000000,14902649,5002876,9893871,0,9899773,5902,868,211,1079,6,0,6,874,211,1085,25559000,23000,6.75,13650,12900,13275.0,566716,7516522400,16350.0,0.231638,1.0
5,제이더블유중외제약(주),1060,2011/09,10.0,24000000,15021777,5002876,10012999,0,10018901,5902,862,241,1103,0,0,6,868,241,1109,38777000,34000,6.67,18800,13900,16350.0,5135907,88630491250,14425.0,-0.117737,0.0
6,제이더블유중외제약(주),1060,2011/12,10.0,24000000,15021777,5002876,10012999,0,10018901,5902,844,241,1085,0,0,0,851,241,1092,51210000,47000,6.75,15150,13700,14425.0,1492799,21992964550,16175.0,0.121317,1.0
7,제이더블유중외제약(주),1060,2012/03,10.0,24000000,15606731,5002876,10595297,0,10603855,8558,840,247,1087,6,0,6,846,247,1093,13891000,13000,5.75,17300,15050,16175.0,2699730,43460675050,13225.0,-0.182380,0.0
8,제이더블유중외제약(주),1060,2012/06,10.0,24000000,15651289,5002876,10639855,0,10648413,8558,778,222,1000,7,0,7,785,222,1007,23502000,23339,5.67,13900,12550,13225.0,562851,7527955000,13650.0,0.032136,1.0
9,제이더블유중외제약(주),1060,2012/09,10.0,24000000,15651289,5002876,10639855,0,10648413,8558,782,230,1012,0,0,6,788,230,1018,36809000,35967,5.50,14050,13250,13650.0,650816,8888507700,14000.0,0.025641,1.0


In [40]:
greencross = greencross.iloc[:-1,:]
daewoong = daewoong.iloc[:-1,:]
yoohan = yoohan.iloc[:-1,:]
hanmi = hanmi.iloc[:-1,:]
#---------- validation ---------------------
donga = donga.iloc[:-1,:]
boogwang = boogwang.iloc[:-1,:]
ilyang = ilyang.iloc[:-1,:]
jw = jw.iloc[:-1,:]

# 가치주로 변수 선정하기

In [41]:
greencross_corr = greencross.corr(method = 'pearson')
df_greencross = pd.DataFrame(greencross_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_greencross.columns = ['지표','3개월후종가']

daewoong_corr = daewoong.corr(method = 'pearson')
df_daewoong = pd.DataFrame(daewoong_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_daewoong.columns = ['지표','3개월후종가']

yoohan_corr = yoohan.corr(method = 'pearson')
df_yoohan = pd.DataFrame(yoohan_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_yoohan.columns = ['지표','3개월후종가']

hanmi_corr = hanmi.corr(method = 'pearson')
df_hanmi = pd.DataFrame(hanmi_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_hanmi.columns = ['지표','3개월후종가']

#---------- validation ---------------------



In [42]:
pd.concat([df_greencross.reset_index(drop = True), df_daewoong.reset_index(drop = True),
           df_yoohan.reset_index(drop = True), df_hanmi.reset_index(drop = True)],axis = 1)

,지표,3개월후종가,지표,3개월후종가,지표,3개월후종가,지표,3개월후종가
0,3개월후종가,1.000000,3개월후종가,1.000000,3개월후종가,1.000000,3개월후종가,1.000000
1,최고가,0.719608,최저가,0.915958,종가,0.858799,최고가,0.905746
2,종가,0.719570,종가,0.913289,최고가,0.858690,종가,0.892107
3,최저가,0.703807,최고가,0.903792,최저가,0.854375,최저가,0.872062
4,합계_기말인원_여(명),0.645558,정규+무기계약_기말인원(명),0.873611,합계_평균근속연수,0.787942,거래대금,0.790489
5,합계_기말인원(명),0.635535,정규+무기계약_기말인원(여)(명),0.847674,자기주식수(주),0.736998,유통주식수 (발행주식의 총수 - 자기주식수)(주),0.782966
6,합계_기말인원_남(명),0.629191,합계_기말인원_여(명),0.710143,거래대금,0.708051,발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주),0.782657
7,자기주식수(주),0.572928,합계_기말인원(명),0.644079,정규+무기계약_기말인원(여)(명),0.596281,현재까지 발행한 주식의 총수(주),0.782657
8,정규+무기계약_기말인원(여)(명),0.537314,정규+무기계약_기말인원(남)(명),0.574512,합계_기말인원_여(명),0.543245,정규+무기계약_기말인원(여)(명),0.714580
9,기간제_기말인원(명),0.490319,유통주식수 (발행주식의 총수 - 자기주식수)(주),0.560006,정규+무기계약_기말인원(명),0.515214,정규+무기계약_기말인원(명),0.713801


In [43]:
list_corr = []
order = 15 # 기업별로 상위 30개 상관관계 변수 추출

for i in range(len(df_greencross.index[:order])):
    list_corr.append(df_greencross.index[:order][i])
    list_corr.append(df_daewoong.index[:order][i])
    list_corr.append(df_yoohan.index[:order][i])
    list_corr.append(df_hanmi.index[:order][i])

variable = pd.Series(list_corr).value_counts()
variable = pd.DataFrame(variable).reset_index()
variable.columns = ['var_index', 'number']

In [44]:
var = []
for i in range(len(variable)):
    var.append(df_greencross[df_greencross.index == variable['var_index'][i]]['지표'].values[0])
var = pd.DataFrame(var)
var.columns = ['var']

In [45]:
corr = pd.concat([variable,var], axis = 1)
corr = corr[['var_index','var','number']]
corr = corr[corr['var'] != '3개월후종가'].reset_index(drop = True)
corr = corr[corr['var'] != '수익률상승하락'].reset_index(drop = True)
corr = corr[corr['var'] != '분기수익률'].reset_index(drop = True)
corr

,var_index,var,number
0,11,정규+무기계약_기말인원(명),4
1,25,거래대금,4
2,23,종가,4
3,22,최저가,4
4,21,최고가,4
5,17,합계_기말인원(명),4
6,16,합계_기말인원_여(명),4
7,10,정규+무기계약_기말인원(여)(명),4
8,15,합계_기말인원_남(명),3
9,8,자기주식수(주),3


# 선정한 변수로 모델 만들기 / validation 에도 적용

In [46]:
deep_greencross = greencross[corr['var'][0:18]]
deep_yoohan = yoohan[corr['var'][0:18]]
deep_daewoong = daewoong[corr['var'][0:18]]
deep_hanmi = hanmi[corr['var'][0:18]]

deep_greencross['수익률상승하락'] = greencross['수익률상승하락']
deep_yoohan['수익률상승하락'] = yoohan['수익률상승하락']
deep_daewoong['수익률상승하락'] = daewoong['수익률상승하락']
deep_hanmi['수익률상승하락'] = hanmi['수익률상승하락']

# ------------- validation ------------------

deep_donga = donga[corr['var'][0:18]]
deep_boogwang = boogwang[corr['var'][0:18]]
deep_ilyang = ilyang[corr['var'][0:18]]
deep_jw = jw[corr['var'][0:18]]

deep_donga['수익률상승하락'] = donga['수익률상승하락']
deep_boogwang['수익률상승하락'] = boogwang['수익률상승하락']
deep_ilyang['수익률상승하락'] = ilyang['수익률상승하락']
deep_jw['수익률상승하락'] = jw['수익률상승하락']

In [47]:
# import re

In [48]:
# 표준화

from sklearn.preprocessing import StandardScaler

scaler_greencross = StandardScaler()
scale_greencross = pd.DataFrame(scaler_greencross.fit_transform(deep_greencross.iloc[:,:-1]))

scaler_yoohan = StandardScaler()
scale_yoohan = pd.DataFrame(scaler_yoohan.fit_transform(deep_yoohan.iloc[:,:-1]))

scaler_daewoong = StandardScaler()
scale_daewoong = pd.DataFrame(scaler_daewoong.fit_transform(deep_daewoong.iloc[:,:-1]))

scaler_hanmi = StandardScaler()
scale_hanmi = pd.DataFrame(scaler_hanmi.fit_transform(deep_hanmi.iloc[:,:-1]))

# 원래 값으로 변환
# scaler_lg.inverse_transform(scale_lg)

# ------------- validation ------------------

from sklearn.preprocessing import StandardScaler

scaler_donga = StandardScaler()
scale_donga = pd.DataFrame(scaler_donga.fit_transform(deep_donga.iloc[:,:-1]))

scaler_boogwang = StandardScaler()
scale_boogwang = pd.DataFrame(scaler_boogwang.fit_transform(deep_boogwang.iloc[:,:-1]))

scaler_ilyang = StandardScaler()
scale_ilyang = pd.DataFrame(scaler_ilyang.fit_transform(deep_ilyang.iloc[:,:-1]))

scaler_jw = StandardScaler()
scale_jw = pd.DataFrame(scaler_jw.fit_transform(deep_jw.iloc[:,:-1]))

In [49]:
# 주가를 합치려고 만듬(마지막 열을 가져오면 됨)
deep_Medicine = pd.concat([deep_greencross,deep_yoohan,deep_daewoong,deep_hanmi], ignore_index=True)

# 표준화한 변수를 합치려고 만듬
scale_Medicine = pd.concat([scale_greencross,scale_yoohan,scale_daewoong,scale_hanmi], ignore_index=True)

# ------------- validation ------------------

# 주가를 합치려고 만듬(마지막 열을 가져오면 됨)
deep_Medicine_validation = pd.concat([deep_donga,deep_boogwang,deep_ilyang,deep_jw], ignore_index=True)

# 표준화한 변수를 합치려고 만듬
scale_Medicine_validation = pd.concat([scale_donga,scale_boogwang,scale_ilyang,scale_jw], ignore_index=True)

In [50]:
# import matplotlib.pyplot as plt
# from matplotlib import font_manager, rc
# import seaborn as sns
# import matplotlib
# font_location = "c:/Windows/fonts/malgun.ttf"
# font_name = font_manager.FontProperties(fname=font_location).get_name()
# matplotlib.rc('font', family=font_name)

In [51]:
# plt.figure(figsize=(20,50))

# sns.heatmap(data = scale_Medicine.corr(), annot = True)
# # plt.savefig('tmp.png')

In [52]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [56]:
# 히든레이어 1개 짜리!!!!

# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility
tf.reset_default_graph()


x_data = scale_Medicine
y_data = deep_Medicine.iloc[:,[-1]]


# parameters
placeholder_num = len(x_data.columns)
learning_rate = 0.001


X = tf.placeholder(tf.float32, [None, placeholder_num])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)


W1 = tf.get_variable("W1", shape=[placeholder_num, 1], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([1]))

hypothesis = tf.sigmoid(tf.matmul(X, W1) + b1)


# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                       tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# train = tf.train.AdamOptimizer(1e-4).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [54]:
# 

# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility
tf.reset_default_graph()


# x_data = scale_Medicine
# y_data = deep_Medicine.iloc[:,[-1]]


# # parameters
# placeholder_num = len(x_data.columns)
# learning_rate = 0.001




# X = tf.placeholder(tf.float32, [None, placeholder_num])
# Y = tf.placeholder(tf.float32, [None, 1])
# keep_prob = tf.placeholder(tf.float32)


# W1 = tf.get_variable("W1", shape=[placeholder_num, 32], initializer=tf.contrib.layers.xavier_initializer())
# b1 = tf.Variable(tf.random_normal([32]))
# L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
# L1 = tf.nn.dropout(L1, keep_prob=keep_prob)


# W2 = tf.get_variable("W2", shape=[32, 64], initializer=tf.contrib.layers.xavier_initializer())
# b2 = tf.Variable(tf.random_normal([64]))
# L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
# L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# W3 = tf.get_variable("W3", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
# b3 = tf.Variable(tf.random_normal([64]))
# L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
# L3 = tf.nn.dropout(L3, keep_prob=keep_prob)


# W4 = tf.get_variable("W4", shape=[64, 32], initializer=tf.contrib.layers.xavier_initializer())
# b4 = tf.Variable(tf.random_normal([32]))
# L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
# L4 = tf.nn.dropout(L4, keep_prob=keep_prob)


# W5 = tf.get_variable("W5", shape=[32, 1], initializer=tf.contrib.layers.xavier_initializer())
# b5 = tf.Variable(tf.random_normal([1]))
# L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

# # hypothesis = tf.matmul(L4, W5) + b5

# # define cost/loss & optimizer

# # Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
# hypothesis = tf.sigmoid(tf.matmul(L4,W5) + b5)

# # cost/loss function
# cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
#                        tf.log(1 - hypothesis))

# train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# # train = tf.train.AdamOptimizer(1e-4).minimize(cost)

# # Accuracy computation
# # True if hypothesis>0.5 else False
# predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
# accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# <span style="color:red"> 가치주로 validation

In [58]:
# Launch graph


graph_cost = []
graph_acc = []
validation_number = 20
#init_state = lstm_cell.zero_state(128, tf.float32)
x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(x_data, y_data, test_size=0.2, random_state=0)
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in tqdm_notebook(range(100001)):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data_train, Y: y_data_train, keep_prob: 0.8})       
        if step % 200 == 0 or step < 10 :
            h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data_test, Y: y_data_test, keep_prob: 1.0})
            graph_cost.append(cost_val)
            graph_acc.append(a)
            print("Step : {} \t Cost : {} \t Acc : {}".format(step, cost_val, a*100))    


Step : 0 	 Cost : 1.0483120679855347 	 Acc : 41.66666567325592
Step : 1 	 Cost : 1.0480575561523438 	 Acc : 41.66666567325592
Step : 2 	 Cost : 1.047803282737732 	 Acc : 41.66666567325592
Step : 3 	 Cost : 1.0475493669509888 	 Acc : 41.66666567325592
Step : 4 	 Cost : 1.0472955703735352 	 Acc : 41.66666567325592
Step : 5 	 Cost : 1.0470421314239502 	 Acc : 41.66666567325592
Step : 6 	 Cost : 1.0467890501022339 	 Acc : 41.66666567325592
Step : 7 	 Cost : 1.0465362071990967 	 Acc : 41.66666567325592
Step : 8 	 Cost : 1.0462837219238281 	 Acc : 41.66666567325592
Step : 9 	 Cost : 1.0460314750671387 	 Acc : 41.66666567325592
Step : 200 	 Cost : 1.002345085144043 	 Acc : 41.66666567325592
Step : 400 	 Cost : 0.9636875987052917 	 Acc : 45.83333432674408
Step : 600 	 Cost : 0.9298399090766907 	 Acc : 45.83333432674408
Step : 800 	 Cost : 0.8994935154914856 	 Acc : 45.83333432674408
Step : 1000 	 Cost : 0.871945321559906 	 Acc : 45.83333432674408
Step : 1200 	 Cost : 0.8467927575111389 	 Acc :

Step : 23800 	 Cost : 0.5585913062095642 	 Acc : 54.16666865348816
Step : 24000 	 Cost : 0.5584530830383301 	 Acc : 54.16666865348816
Step : 24200 	 Cost : 0.558317244052887 	 Acc : 54.16666865348816
Step : 24400 	 Cost : 0.5581836104393005 	 Acc : 54.16666865348816
Step : 24600 	 Cost : 0.5580520033836365 	 Acc : 54.16666865348816
Step : 24800 	 Cost : 0.5579226613044739 	 Acc : 54.16666865348816
Step : 25000 	 Cost : 0.5577952265739441 	 Acc : 54.16666865348816
Step : 25200 	 Cost : 0.5576699376106262 	 Acc : 54.16666865348816
Step : 25400 	 Cost : 0.5575465559959412 	 Acc : 54.16666865348816
Step : 25600 	 Cost : 0.5574251413345337 	 Acc : 54.16666865348816
Step : 25800 	 Cost : 0.5573055148124695 	 Acc : 54.16666865348816
Step : 26000 	 Cost : 0.5571878552436829 	 Acc : 54.16666865348816
Step : 26200 	 Cost : 0.5570719242095947 	 Acc : 54.16666865348816
Step : 26400 	 Cost : 0.5569577217102051 	 Acc : 54.16666865348816
Step : 26600 	 Cost : 0.5568453669548035 	 Acc : 54.16666865348

Step : 48200 	 Cost : 0.550233781337738 	 Acc : 58.33333134651184
Step : 48400 	 Cost : 0.5501995086669922 	 Acc : 58.33333134651184
Step : 48600 	 Cost : 0.5501655340194702 	 Acc : 58.33333134651184
Step : 48800 	 Cost : 0.5501317977905273 	 Acc : 58.33333134651184
Step : 49000 	 Cost : 0.5500982403755188 	 Acc : 58.33333134651184
Step : 49200 	 Cost : 0.5500650405883789 	 Acc : 58.33333134651184
Step : 49400 	 Cost : 0.5500319600105286 	 Acc : 58.33333134651184
Step : 49600 	 Cost : 0.5499991774559021 	 Acc : 58.33333134651184
Step : 49800 	 Cost : 0.5499665141105652 	 Acc : 58.33333134651184
Step : 50000 	 Cost : 0.5499342083930969 	 Acc : 58.33333134651184
Step : 50200 	 Cost : 0.549902081489563 	 Acc : 58.33333134651184
Step : 50400 	 Cost : 0.5498701930046082 	 Acc : 58.33333134651184
Step : 50600 	 Cost : 0.5498384833335876 	 Acc : 58.33333134651184
Step : 50800 	 Cost : 0.5498068928718567 	 Acc : 58.33333134651184
Step : 51000 	 Cost : 0.5497755408287048 	 Acc : 58.333331346511

Step : 72600 	 Cost : 0.5471659898757935 	 Acc : 58.33333134651184
Step : 72800 	 Cost : 0.5471463799476624 	 Acc : 58.33333134651184
Step : 73000 	 Cost : 0.547126829624176 	 Acc : 58.33333134651184
Step : 73200 	 Cost : 0.5471072793006897 	 Acc : 58.33333134651184
Step : 73400 	 Cost : 0.5470877885818481 	 Acc : 58.33333134651184
Step : 73600 	 Cost : 0.5470682978630066 	 Acc : 58.33333134651184
Step : 73800 	 Cost : 0.5470488667488098 	 Acc : 58.33333134651184
Step : 74000 	 Cost : 0.5470294952392578 	 Acc : 58.33333134651184
Step : 74200 	 Cost : 0.5470101833343506 	 Acc : 58.33333134651184
Step : 74400 	 Cost : 0.5469909310340881 	 Acc : 58.33333134651184
Step : 74600 	 Cost : 0.5469716191291809 	 Acc : 58.33333134651184
Step : 74800 	 Cost : 0.5469524264335632 	 Acc : 58.33333134651184
Step : 75000 	 Cost : 0.5469333529472351 	 Acc : 58.33333134651184
Step : 75200 	 Cost : 0.546914279460907 	 Acc : 58.33333134651184
Step : 75400 	 Cost : 0.5468952655792236 	 Acc : 58.333331346511

Step : 97000 	 Cost : 0.5450408458709717 	 Acc : 58.33333134651184
Step : 97200 	 Cost : 0.5450250506401062 	 Acc : 58.33333134651184
Step : 97400 	 Cost : 0.5450093150138855 	 Acc : 58.33333134651184
Step : 97600 	 Cost : 0.5449936389923096 	 Acc : 58.33333134651184
Step : 97800 	 Cost : 0.5449779629707336 	 Acc : 58.33333134651184
Step : 98000 	 Cost : 0.5449622273445129 	 Acc : 58.33333134651184
Step : 98200 	 Cost : 0.5449466705322266 	 Acc : 54.16666865348816
Step : 98400 	 Cost : 0.5449311137199402 	 Acc : 54.16666865348816
Step : 98600 	 Cost : 0.5449156165122986 	 Acc : 54.16666865348816
Step : 98800 	 Cost : 0.5449000000953674 	 Acc : 54.16666865348816
Step : 99000 	 Cost : 0.5448845028877258 	 Acc : 54.16666865348816
Step : 99200 	 Cost : 0.5448690056800842 	 Acc : 54.16666865348816
Step : 99400 	 Cost : 0.5448535084724426 	 Acc : 54.16666865348816
Step : 99600 	 Cost : 0.5448381304740906 	 Acc : 54.16666865348816
Step : 99800 	 Cost : 0.544822633266449 	 Acc : 54.16666865348

In [59]:
print('모델 최고 정확도: ', max(graph_acc) * 100 , '%')

모델 최고 정확도:  62.5 %


# <span style="color:red"> 비 가치주로 validation

In [29]:
# validation 기업!!!!!!!!

# Launch graph

validation_number = 0



x_val_data = scale_Medicine_validation
y_val_data = deep_Medicine_validation.iloc[:,[-1]]

# x_val_data = scale_IT_validation
# y_val_data = deep_IT_validation.iloc[:,[-1]]

with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in tqdm_notebook(range(20001)):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data, keep_prob: 1.0})       
        if step % 200 == 0 or step < 10 :
            h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_val_data.iloc[:,:], Y: y_val_data.iloc[:,:], keep_prob: 1.0})
            print("Step : {} \t Cost : {} \t Acc : {}".format(step, cost_val, a*100))     


Step : 0 	 Cost : 1.0436207056045532 	 Acc : 46.72897160053253
Step : 1 	 Cost : 1.0431870222091675 	 Acc : 46.72897160053253
Step : 2 	 Cost : 1.042754054069519 	 Acc : 46.72897160053253
Step : 3 	 Cost : 1.042321801185608 	 Acc : 46.72897160053253
Step : 4 	 Cost : 1.0418903827667236 	 Acc : 46.72897160053253
Step : 5 	 Cost : 1.0414594411849976 	 Acc : 46.72897160053253
Step : 6 	 Cost : 1.0410293340682983 	 Acc : 46.72897160053253
Step : 7 	 Cost : 1.0405999422073364 	 Acc : 46.72897160053253
Step : 8 	 Cost : 1.0401712656021118 	 Acc : 46.72897160053253
Step : 9 	 Cost : 1.0397433042526245 	 Acc : 46.72897160053253
Step : 200 	 Cost : 0.9697245359420776 	 Acc : 47.663551568984985
Step : 400 	 Cost : 0.9160428643226624 	 Acc : 47.663551568984985
Step : 600 	 Cost : 0.8759719729423523 	 Acc : 45.79439163208008
Step : 800 	 Cost : 0.8449472784996033 	 Acc : 44.859811663627625
Step : 1000 	 Cost : 0.8199467062950134 	 Acc : 43.92523467540741
Step : 1200 	 Cost : 0.7990536093711853 	 A

In [31]:
print('모델 최고 정확도: ', max(graph_acc) * 100 , '%')

NameError: name 'graph_acc' is not defined

In [ ]:
# pd.DataFrame(y_data.iloc[-validation_number:,:].values, c.ravel()).reset_index()
pd.DataFrame({'real':list(y_data.iloc[-validation_number:,:].values.ravel()),'predict': list(c1.ravel())})

In [ ]:
h1